In [ ]:
####################################################################################
# Name: Pantip Scraping 
# Description: Get post date, link, topic name, and tags
# Version:
#   09/20/2016 SN: Initial version
#   09/21/2016 SN: Update to handle and skip either missing page or deleted page
####################################################################################

from urllib.request import urlopen
from lxml import html
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

def get_pantip_scrape(df, s):
    page_init = 30000000
    page_num = page_init + s
    site = "http://pantip.com/topic/"+ str(page_num) + ".html"
    if s%100 == 0:
        print("Now reading: " + site)
    response = urlopen(site)
    html = response.read().decode('utf-8')
    soup = BeautifulSoup(html, "html.parser")
    notFoundPage = len(re.findall('ไม่พบหน้าที่คุณต้องการ',html))
    deletePage = len(re.findall('เจ้าของข้อความแจ้งลบค่ะ', html))
    print(notFoundPage, deletePage)
    if notFoundPage == 0 and deletePage == 0:
        # Get topic create time
        tmp_post_time = soup.find("abbr", class_="timeago")
        post_time_list = re.findall(r'(?<=data-utime=\")[^\"]*', str(tmp_post_time))
        # Get header; alternatively can use soup.title.contents
        title_list = soup.find("h2", class_="display-post-title").contents
        # Get page tags
        tag_list =[]
        for tag in soup.findAll("a", {"class" : "tag-item"} ):
            tmp_tag = tag.contents
            tmp_tag_str = str(tmp_tag)
            tmp_tag_str = tmp_tag_str.strip("['']")
            tag_list.append(tmp_tag_str)
        tag_list = list(set(tag_list))

        df2 = pd.DataFrame( {'a_index': s,
                             'url' : site,
                            'title': title_list,
                            'create_time' : post_time_list,
                            })   
        df2.ix[df.index.get_values(), 'tag'] = ', '.join(tag_list)
        df = df.append(df2)
        return df
    else:
        return df

columns = ['create_time', 'title', 'url', 'tag']
df = pd.DataFrame(columns=columns)
df = df.fillna(0)
for i in range(100,1000):
    df = get_pantip_scrape(df, i)

Now reading: http://pantip.com/topic/30000100.html
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 1
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 1
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 1
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
Now reading: http://pantip.com/topic/30000200.html


In [ ]:
df.head()

In [9]:
df.to_csv("pantip_scraping.csv", sep= ',')

In [11]:
101 %100

1